In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
import ndreg
from ndreg import preprocessor, util, plotter
import SimpleITK as sitk

# Register the data to the ARA

In [2]:
params = {
    # input image path
    'image_path': '../Thy1eYFP_Control_9.tiff',
    # voxel spacing is in mm and corresponds to (x, y, z) spacing
    'image_spacing': (0.04128, 0.04128, 0.04128),
    'image_orientation': 'rpi',
    # the modality can be 'lavision' or 'colm'
    'image_modality': 'lavision',
    'atlas_spacing': (0.05, 0.05, 0.05),
    'atlas_path': '../ARA_50um.tiff',
}

In [3]:
img = util.imgRead(params['image_path'])
img.SetSpacing(params['image_spacing'])
atlas = util.imgRead(params['atlas_path'])
atlas.SetSpacing(params['atlas_spacing'])

In [4]:
img_p = preprocessor.preprocess_brain(img, 
                                      params['atlas_spacing'], 
                                      params['image_modality'],
                                      params['image_orientation'])

In [5]:
atlas_registered = ndreg.register_brain(atlas, img_p)


Step 0: alpha=0.05, beta=0.05, scale=0.0625
	E, E_velocity, E_rate, E_image (E_image %), LearningRate
0.	85226.1, 89.2659, 0, 85136.9 (96.8675%), 2.750000e-03
1.	80573.9, 88.3305, 0, 80485.6 (91.5753%), 1.512500e-03
2.	74680.7, 124.017, 0, 74556.6 (84.8295%), 1.663750e-03
3.	72149.4, 157.015, 0, 71992.4 (81.9119%), 1.830125e-03
4.	70143, 213.898, 0, 69929.1 (79.5643%), 2.013138e-03
5.	68311.6, 280.149, 0, 68031.5 (77.4052%), 2.214451e-03
6.	67836.8, 327.152, 0, 67509.6 (76.8114%), 1.217948e-03
7.	66489.6, 375.896, 0, 66113.7 (75.2231%), 1.339743e-03
8.	66300.1, 440.192, 0, 65859.9 (74.9344%), 1.473717e-03
9.	65347.6, 510.434, 0, 64837.1 (73.7707%), 1.621089e-03
10.	64436.2, 552.761, 0, 63883.5 (72.6857%), 8.915990e-04
11.	63019.6, 599.2, 0, 62420.4 (71.021%), 9.807589e-04
12.	62934.8, 653.686, 0, 62281.2 (70.8626%), 1.078835e-03
13.	62139.4, 719.298, 0, 61420.1 (69.8828%), 1.186718e-03
14.	61947.6, 787.936, 0, 61159.7 (69.5866%), 1.305390e-03
15.	60819.2, 828.633, 0, 59990.6 (68.2564%

# Detect cells in chunks

In [4]:
path_to_chunks = './test_data_cobalt/cell_detection_chunks/'

In [5]:
from cell_detector import *

In [6]:
def detect_all_cells(chunks, csv_path): 
    threshold=1.0
    jobs=1
    verbose=0
    max_val_ = get_max_intensity(chunks)
    paired_files = get_paired_files(chunks)
    for i in tqdm(paired_files, desc='detecting cells...'):
        detect_cells(i,csv_path,threshold, max_val=max_val_, verbose=verbose)
    centroids = get_cells_from_csv(csv_path)
    centroids_t = prune_clusters(centroids[:,:3], verbose=verbose)
    write_list_to_csv(centroids_t, csv_path, open_mode='w')

In [ ]:
detect_all_cells(path_to_chunks, './test1.csv')

detecting cells...:   0%|          | 0/1 [00:00<?, ?it/s]

# quantification of results